In [13]:
# -*- coding: utf-8 -*-
# Author: Olivier Grisel <olivier.grisel@ensta.org>
#         Lars Buitinck <L.J.Buitinck@uva.nl>
#         Chyi-Kwei Yau <chyikwei.yau@gmail.com>
# License: BSD 3 clause

from __future__ import print_function
from time import time

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.datasets import fetch_20newsgroups

import numpy as np
import logging
import pandas as pd

In [40]:
n_samples = 2000
n_features = 1000
n_topics = 100
n_top_words = 5

In [41]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

In [2]:
# Load the 20 newsgroups dataset and vectorize it. We use a few heuristics
# to filter out useless terms early on: the posts are stripped of headers,
# footers and quoted replies, and common English words, words occurring in
# only one document or in at least 95% of the documents are removed.

print("Loading dataset...")
t0 = time()
dataset = fetch_20newsgroups(shuffle=True, random_state=1,
                             remove=('headers', 'footers', 'quotes'))
data_samples = dataset.data
print("done in %0.3fs." % (time() - t0))

Loading dataset...
done in 2.022s.


In [11]:
print (data_samples[3])

Notwithstanding all the legitimate fuss about this proposal, how much
of a change is it?  ATT's last product in this area (a) was priced over
$1000, as I suspect 'clipper' phones will be; (b) came to the customer 
with the key automatically preregistered with government authorities. Thus,
aside from attempting to further legitimize and solidify the fed's posture,
Clipper seems to be "more of the same", rather than a new direction.
   Yes, technology will eventually drive the cost down and thereby promote
more widespread use- but at present, the man on the street is not going
to purchase a $1000 crypto telephone, especially when the guy on the other
end probably doesn't have one anyway.  Am I missing something?
   The real question is what the gov will do in a year or two when air-
tight voice privacy on a phone line is as close as your nearest pc.  That
has got to a problematic scenario for them, even if the extent of usage
never surpasses the 'underground' stature of PGP.


In [30]:
stop_words = ['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', \
             'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', \
             'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', \
             'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', \
             'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', \
             'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', \
             'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', \
             'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', \
             'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', \
             'много', 'разве', 'три', 'эту', 'моя', 'впрочем', 'хорошо', 'свою', 'этой', 'перед', 'иногда', 'лучше', 'чуть', \
             'том', 'нельзя', 'такой', 'им', 'более', 'всегда', 'конечно', 'всю', 'между', 'ру']

In [31]:
%%time
data=pd.read_csv('data-w2v/query-count-direct-2015-10.txt', sep='\t', names=['query', 'wordstat', 'click'])
data.dropna(subset=['query'], inplace=True)
data.drop('click', axis=1, inplace=True)


Wall time: 34.7 s


In [51]:
%%time
data1=data[:100000].copy()
data_samples=data1['query'].tolist()

Wall time: 4 ms


In [52]:
print (data_samples[23])

сбербанк


In [53]:
# Use tf-idf features for NMF.
print("Extracting tf-idf features for NMF...")
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features=n_features,
                                   stop_words=stop_words)
t0 = time()
tfidf = tfidf_vectorizer.fit_transform(data_samples)
print("done in %0.3fs." % (time() - t0))


Extracting tf-idf features for NMF...
done in 0.698s.


In [54]:
# Use tf (raw term count) features for LDA.
print("Extracting tf features for LDA...")
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=n_features,
                                stop_words=stop_words)
t0 = time()
tf = tf_vectorizer.fit_transform(data_samples)
print("done in %0.3fs." % (time() - t0))


Extracting tf features for LDA...
done in 0.702s.


In [55]:
# Fit the NMF model
print("Fitting the NMF model with tf-idf features, n_samples=%d and n_features=%d..." % (n_samples, n_features))
t0 = time()
nmf = NMF(n_components=n_topics, random_state=1, alpha=.1, l1_ratio=.5).fit(tfidf)
#exit()
print("done in %0.3fs." % (time() - t0))

Fitting the NMF model with tf-idf features, n_samples=2000 and n_features=1000...
done in 190.491s.


In [58]:
##Шляпа какая то вызывает ошибку
print("\nTopics in NMF model:")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)


Topics in NMF model:
Topic #0:
онлайн сбербанк читать бизнес табло
Topic #1:
сайт официальный санаторий знакомств фк
Topic #2:
ru http mail rambler mos
Topic #3:
погода дня неделю дней 14
Topic #4:
ру майл налог дром знакомства
Topic #5:
порно русское ролики hd зрелые
Topic #6:
на русском дня дону неделю
Topic #7:
новости последние украины украина слухи
Topic #8:
для детей девочек кухни дачи
Topic #9:
как приготовить сделать правильно капусту
Topic #10:
скачать торрент через музыку майнкрафт
Topic #11:
инструкция применению мазь форте таблетки
Topic #12:
магазин интернет детский одежды мир
Topic #13:
смотреть качестве хорошем онлайн 2013
Topic #14:
видео домашнее майнкрафт про частное
Topic #15:
фото девушек девушки женщин ню
Topic #16:
самара гис дубль шахматы валют
Topic #17:
авито область пробегом продажа квартиры
Topic #18:
купить квартиру магазине билет айфон
Topic #19:
com ua http vk русском
Topic #20:
из блюда поделки салат тыквы
Topic #21:
форум жк кубань парк дача
Topic #22:


(n_topics=10, doc_topic_prior=None, topic_word_prior=None, learning_method='online', learning_decay=0.7, learning_offset=10.0, max_iter=10, batch_size=128, evaluate_every=-1, total_samples=1000000.0, perp_tol=0.1, mean_change_tol=0.001, max_doc_update_iter=100, n_jobs=1, verbose=0, random_state=None)[source]


In [56]:
print("Fitting LDA models with tf features, n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=5,
                                learning_method='online', learning_offset=50.,
                                random_state=0, evaluate_every=10)
t0 = time()
lda.fit(tf)
print("done in %0.3fs." % (time() - t0))

Fitting LDA models with tf features, n_samples=2000 and n_features=1000...
done in 52.303s.


In [57]:
print("\nTopics in LDA model:")
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)


Topics in LDA model:
Topic #0:
жизнь rambler чат тверь личная
Topic #1:
форум где санаторий серия слова
Topic #2:
ремонт лада фз of квартир
Topic #3:
новгород нижний сказки словарь великий
Topic #4:
порно фото русское porno hd
Topic #5:
акции гис юлия три дубль
Topic #6:
магазин интернет санкт петербург через
Topic #7:
ру википедия при стихи аптека
Topic #8:
авито самара краснодар казань омск
Topic #9:
валют курсы ню 12 банках
Topic #10:
центр голос между отопления эль
Topic #11:
по класс гдз футболу математике
Topic #12:
как любовь не приготовить сделать
Topic #13:
русский google переводчик язык английский
Topic #14:
воронеж новые своими руками во
Topic #15:
вести фм закон поликлиника марина
Topic #16:
симптомы обувь показания воды планета
Topic #17:
это воин рассказы эротические чудо
Topic #18:
карты яндекс деньги гта музыка
Topic #19:
игры школа бесплатные флеш канобу
Topic #20:
каталог екатеринбург товаров осаго мебели
Topic #21:
ком компания чемпионат линии андроид
Topic #22:
вид

In [60]:
lda.score(tf)

-805628.87547947315

In [59]:
lda.perplexity(tf, doc_topic_distr=None, sub_sampling=False)

2552.0196848587475